In [1]:
! pip install selenium
! pip install beautifulsoup4

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from collections import defaultdict

In [4]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections
import pickle

In [22]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_path = "./chromedriver"

In [23]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [24]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)

In [25]:
comment_info_xpath = "//div[@id='main']"
show_more_button_xpath = "//span[@class='more-button style-scope ytd-comment-renderer']"
number_of_comments = "//h2[@id='count']/yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"
# comment_total_amount_xpath = "//yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"

In [26]:
df = pd.read_excel('DataSource_80clips.xlsx')
# remove the "who" and sort the table with "comedian"
df = df.drop('Who?', axis=1)
df = df.sort_values('Comedian')
df.tail()

,Comedian,Title,Link of the transcript,Transcript (CONTENT!!!!),Video link
41,Urzila Carlson:,What No One Tells You About Homemade Porn,https://scrapsfromtheloft.com/2020/07/19/urzil...,"It makes no sense. Now, I know… I know I need ...",https://www.youtube.com/watch?v=AW5FoxJ-C5w&ab...
27,Vir Das,Losing It (2018),https://scrapsfromtheloft.com/2018/12/15/vir-d...,Every time a woman in India wears something re...,https://www.youtube.com/watch?v=qtswR_SW8S0&ab...
12,Vir Das,Losing It,https://scrapsfromtheloft.com/2018/12/15/vir-d...,I lost 80% of my religion this year. And I thi...,https://www.youtube.com/watch?v=wE9OuVw2XI8
17,Whitney Cummings,Marriage & Sex,https://scrapsfromtheloft.com/2017/12/25/whitn...,Love makes you think about things you never th...,https://www.youtube.com/watch?v=T6GstyTrLl0
14,Whitney Cummings,I'm Your Girlfriend: Women Paying,https://scrapsfromtheloft.com/2017/12/25/whitn...,We’re living in a fascinating time. We’re livi...,https://www.youtube.com/watch?v=A1OKxp91B2M


In [27]:
all_urls = df["Video link"]
comedians = df.Comedian

In [28]:
len(all_urls)

80

In [29]:
len(comedians)

80

In [17]:
web_list = []
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:    
    for url in all_urls:
        all_comments_info = []        
        prev_offest = -1
        offset = 0
        
        print(f"\nUrl: {url}")
        driver.get(url)
        # Scroll until the number of comments shows up
        while driver.wait_elements_xpath(number_of_comments, timeout=1, verbose=False) == None:
            driver.execute_script("scrollBy(0, 50)")
            
        # Scarpe comments until all the comments are collected
        while offset != prev_offest:
            prev_offest = offset
            driver.execute_script("scrollBy(0, 1000)")
            time.sleep(2)
            offset = driver.execute_script("return window.pageYOffset;");
            all_comments_info = driver.wait_elements_xpath(comment_info_xpath)
            print("", end=".")
        # Get whole html for us to find sth later
        html = driver.page_source    
        whole_page = bs(html,"html.parser")
        web_list.append(whole_page)


Url: https://www.youtube.com/watch?v=gvSqKSkRFuM&ab_channel=HBO
..........................................
Url: https://www.youtube.com/watch?v=obphkbG5FIo&ab_channel=DryBarComedy
..........................
Url: https://www.youtube.com/watch?v=EmzPp33kl7o
............
Url: https://www.youtube.com/watch?v=c_KmV-hquvc&ab_channel=NetflixIsAJoke
...........
Url: https://www.youtube.com/watch?v=oTHpQjE1VQE&ab_channel=NetflixIsAJoke
.................................
Url: https://www.youtube.com/watch?v=lWA-xsAVzgE
.............
Url: https://www.youtube.com/watch?v=X8Ug3-_S174&ab_channel=NetflixIsAJoke
...........................................
Url: https://www.youtube.com/watch?v=io2d_cpoLDg&ab_channel=NetflixIsAJoke
.......................................
Url: https://www.youtube.com/watch?v=Oa6X6ufoiBM&ab_channel=GrettaVonPissflaps
.........................................
Url: https://www.youtube.com/watch?v=RRLf-NaMfLc&list=PLVXN0tFUddVTkbmqYUTS_OCkMF2ab-NpX&index=4&ab_channel=dncZTV
.

In [18]:
len(web_list)

80

In [29]:
# get clips title
def get_titles(web_list):
    clip_title = []
    for webpage in web_list:
        title = webpage.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        clip_title.append(title)
    return clip_title

In [68]:
def html_to_list(web_list):
    df = pd.DataFrame(columns=['Title','Thumbsup', 'Name','Comment'])    
    for web in web_list:        
        # title
        title = web.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        
        comments = web.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
        comments_names_list = []
        comments_contents_list = []
        comments_thumbsup_list = []
            
        for comment in comments:
            row = {}  
            row['Title'] = title
            # reviewer's thumbsup amount
            thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
            new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
            row['Thumbsup'] = int(eval(new_thumbsup))
            # reviewer's name
            name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()        
            row['Name'] = name.strip('\n ')
            # reviewer's comment
            content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
            row['Comment'] = content
            # append to dataframe
            df = df.append(row, ignore_index=True)
    
    return df

In [70]:
df = html_to_list(web_list)
df

,Title,Thumbsup,Name,Comment
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn..."
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t..."
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ..."
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄
...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ..."


In [71]:
df.to_csv('backup_scrape_result_80.csv',index=False)

In [72]:
df.to_pickle('./pickles/scrape_result_80.pkl')

In [80]:
df_name = df.groupby(["Name"], sort=True).agg(['count']).Title.sort_values('count', ascending=False)
df_name.head(100)

,count
Name,
whitepride1004,31
Sumukh,7
Michael,6
Green Envy,6
Audrey New,6
...,...
Bi- Han,3
Lydia Waweru-Morgan,3
James Bond,3


# Insert Commedians back to the dataframe

In [40]:
df = pd.read_pickle('./pickles/scrape_result_80.pkl')
df

,Title,Thumbsup,Name,Comment
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn..."
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t..."
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ..."
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄
...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ..."


In [41]:
unique_titles = {}
# find all unique titles (cannot use set because it will change its order)
for t in df.Title:
    if t not in unique_titles:
        unique_titles[t] = 1
unique_titles = list(unique_titles.keys())

In [42]:
df['comedian_id'] = [unique_titles.index(t) for t in df.Title]
df

,Title,Thumbsup,Name,Comment,comedian_id
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn...",0
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...,0
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t...",0
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ...",0
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄,0
...,...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...,79
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...,79
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...,79
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ...",79


In [44]:
# Append the comedians
df['Comedians'] = [comedians[idx] for idx in temp_df['comedian_id']]
df

,Title,Thumbsup,Name,Comment,comedian_id,Comedians
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn...",0,Kevin Hart
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...,0,Kevin Hart
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t...",0,Kevin Hart
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ...",0,Kevin Hart
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄,0,Kevin Hart
...,...,...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...,79,Arsenio Hall
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...,79,Arsenio Hall
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...,79,Arsenio Hall
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ...",79,Arsenio Hall


In [39]:
df = df.drop(['Thumbsup','Name','Comment'], axis=1)
grouped_df

,Title,comedian_id
0,"""3 Girls? We Don't Have Time"" | Pete Davidson SMD",132
1,"""Don't Lose Your Accent / Learning Accents"" - ...",20
2,"""Flying Into America"" - Trevor Noah - (Lost In...",60
3,"""Lost Keys"" | Jo Koy : Lights Out",20
4,"""Rice is Rice"" | Jo Koy : Live from Seattle",60
...,...,...
75,When You're Late To Everything. Andy Woodhull,220
76,Whitney Cummings - I Love You - Marriage & Sex,135
77,Whitney Cummings | I'm Your Girlfriend: Women ...,470
78,Why Arsenio Checks His Phone Before His Wife C...,46


In [34]:
df.to_pickle('./pickles/scrape_result_80_with_comedians.pkl')

# User-Comedian Matrix

In [37]:
df

,Comedian,Title,Link of the transcript,Transcript (CONTENT!!!!),Video link
51,AMY YOUSSEF,FEELINGS,https://scrapsfromtheloft.com/2020/06/01/ramy-...,NaN,https://www.youtube.com/watch?v=gvSqKSkRFuM&ab...
73,ANDY WOODHULL,YOU’LL ALWAYS BE LATE (2019),https://scrapsfromtheloft.com/2019/12/15/andy-...,NaN,https://www.youtube.com/watch?v=obphkbG5FIo&ab...
19,Amy Schumer,Porn Endings,https://scrapsfromtheloft.com/2017/09/08/amy-s...,"Well, I like to watch porn. Like, what girls i...",https://www.youtube.com/watch?v=EmzPp33kl7o
28,Arsenio Hall,Smart and Classy (2019),https://scrapsfromtheloft.com/2019/10/30/arsen...,Everything’s changing. This has changed a lot....,https://www.youtube.com/watch?v=c_KmV-hquvc&ab...
79,Arsenio Hall,Smart and Classy (2019),https://scrapsfromtheloft.com/2019/10/30/arsen...,NaN,https://www.youtube.com/watch?v=oTHpQjE1VQE&ab...
...,...,...,...,...,...
41,Urzila Carlson:,What No One Tells You About Homemade Porn,https://scrapsfromtheloft.com/2020/07/19/urzil...,"It makes no sense. Now, I know… I know I need ...",https://www.youtube.com/watch?v=AW5FoxJ-C5w&ab...
27,Vir Das,Losing It (2018),https://scrapsfromtheloft.com/2018/12/15/vir-d...,Every time a woman in India wears something re...,https://www.youtube.com/watch?v=qtswR_SW8S0&ab...
12,Vir Das,Losing It,https://scrapsfromtheloft.com/2018/12/15/vir-d...,I lost 80% of my religion this year. And I thi...,https://www.youtube.com/watch?v=wE9OuVw2XI8
17,Whitney Cummings,Marriage & Sex,https://scrapsfromtheloft.com/2017/12/25/whitn...,Love makes you think about things you never th...,https://www.youtube.com/watch?v=T6GstyTrLl0


In [35]:
df_clean = df[['Name', 'Comedians']]
df_clean['Comedians'] = df_clean.Comedians.apply(lambda x: x.lower().strip())
df_clean

KeyError: "None of [Index(['Name', 'Comedians'], dtype='object')] are in the [columns]"

In [36]:
df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(20)

NameError: name 'df_clean' is not defined

In [107]:
df_clean['duplicate'] = df_clean.duplicated()
df_clean = df_clean[~df_clean['duplicate']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [108]:
df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(20)

,Comedians,duplicate
Name,,
Audrey New,6,6
Sumukh,6,6
Whyinem,6,6
Adam Szasz,5,5
Michael,5,5
matthew styles,5,5
Wuft Chan,5,5
Green Envy,5,5
Panos kanos,4,4


In [109]:
# Select the top 100 users 
users = df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(100).index
users

Index(['Audrey New', 'Sumukh', 'Whyinem', 'Adam Szasz', 'Michael',
       'matthew styles', 'Wuft Chan', 'Green Envy', 'Panos kanos', 'A',
       'R. A.', 'Ian Crouch', 'Michiko Manalang', 'Tyrone Frye',
       'Jennifer Bates', 'John Smith', 'Xfirex Xbites', 'Papa Grounds', 'P V',
       'Keonne Williams', 'Kevin Smith', 'black bird', 'Anton Nym', 'L K',
       'Lxlno', 'Vikram Singh', 'Dee Cohen', 'EaST CoAsT MaCHete',
       'brainflash1',
       'Comedy Central Stand-Up\n\n\n\n      \n    Comedy Central Stand-Up',
       'John Doe', 'Jay Jay', 'ADEDOTUN AKANDE', 'Remedy',
       'John Paul Lafferty', 'Allison Linner', 'rudeboymon', 'Sam',
       'Tragoudistros.MPH', 'Lydia Waweru-Morgan', 'Anna markandan', 'A M',
       'A Z', 'yew 2oob', 'That Guy', 'Bi- Han', 'KiraAsakura14',
       'Cynthia Copland', 'Kurt Sudheim', 'Andrew Wiggin', 'A B',
       'Glenn Davey', 'Tha chocolate god', 'Andrew', 'xirsamoht x',
       'SoundSharp Music Ltd Wade Sharp', 'R B', 'Bonafide Slacker', 'J',

In [120]:
unique_comedians = list(set(df_clean.Comedians))

In [132]:
df_ucm = pd.DataFrame(0, index=users, columns=unique_comedians)
df_ucm

,iliza shlesinger,kenny sebastian,kevin hart,amy schumer,andy woodhull,jimmy o. yang,george lopez,russell peters,dave chappelle,whitney cummings,...,ronny chieng,chris rock:,eric andre,jerry seinfeld,patton oswalt,chris d’elia,hasan minhaj,urzila carlson:,jo koy,aziz ansari
Name,,,,,,,,,,,,,,,,,,,,,
Audrey New,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sumukh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Whyinem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam Szasz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Michael,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
paladro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacob Erickson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yootub Apesta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
df_ucm.to_pickle('./pickles/user_comedian_matrix.pkl')

In [130]:
# The way to assign value into the dataframe
df_ucm.loc['Michael', 'andy woodhull'] = 10000000

In [131]:
df_ucm

,iliza shlesinger,kenny sebastian,kevin hart,amy schumer,andy woodhull,jimmy o. yang,george lopez,russell peters,dave chappelle,whitney cummings,...,ronny chieng,chris rock:,eric andre,jerry seinfeld,patton oswalt,chris d’elia,hasan minhaj,urzila carlson:,jo koy,aziz ansari
Name,,,,,,,,,,,,,,,,,,,,,
Audrey New,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sumukh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Whyinem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam Szasz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Michael,0,0,0,0,10000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
paladro,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacob Erickson,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Yootub Apesta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
